# Fraud Detection

## Data Analysis

In [ ]:
import glob
import pandas as pd
import numpy as np

input_filenames = sorted(glob.glob('fraud_detection/*.Inputs'))
target_filenames = sorted(glob.glob('fraud_detection/*.Targets'))

df = pd.DataFrame()
for input_file, target_file in zip(input_filenames, target_filenames):
    df = df.append(pd.read_csv(input_file))
    df['fraud'] = pd.read_csv(target_file)

df.head()

### Dataset Profiling Report

In [ ]:
# import pandas_profiling

# profile = pandas_profiling.ProfileReport(df)
# profile.to_file(output_file="ProfilingResult.html")

In [ ]:
# profile

### Rows containing null values

In [ ]:
df[pd.isnull(df).any(axis=1)]

## Data Preprocessing

Drop highly correlated columns

In [ ]:
clean_df = df.drop(columns=['hour2', 'domain1', 'total'])
clean_df.head()

In [ ]:
clean_df = clean_df.dropna()
clean_df = clean_df.drop_duplicates()

In [ ]:
clean_df['state1'] = clean_df['state1'].astype('category')
clean_df['state1'] = clean_df['state1'].cat.codes

clean_df.head()

In [ ]:
clean_df.shape

## Split dataset

In [ ]:
from sklearn.model_selection import train_test_split

X = clean_df.drop(columns=['fraud'])
y = clean_df['fraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

In [ ]:
X_train

In [ ]:
# import pandas_profiling

# profile = pandas_profiling.ProfileReport(df_train)
# profile.to_file(output_file="PreprocessedProfilingResult.html")

## Oversampling & Undersampling

### Oversampling Approach

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(ratio='minority')
X_train_oversampling = X_train
y_train_oversampling = y_train
X_res, y_res = smote.fit_resample(X_train_oversampling, y_train_oversampling)

dataset_res = []
for i in range(len(y_res)):
    dataset_res.append(np.append(X_res[i], y_res[i]))

dataset_res = np.asarray(dataset_res)
dataset_res.shape
dataset_res

In [ ]:
dataset_res[0]

In [ ]:
columns = X.columns.tolist()
columns.append('fraud')

oversampled_df = pd.DataFrame(dataset_res, columns=columns)
oversampled_df

In [ ]:
# import pandas_profiling

# profile = pandas_profiling.ProfileReport(oversampled_df)
# profile.to_file(output_file="OversampledProfilingResult.html")

In [ ]:
oversampled_df['fraud'].value_counts()

### Undersampling Approach

In [ ]:
from imblearn.under_sampling import RepeatedEditedNearestNeighbours 
from sklearn.datasets import make_classification
from collections import Counter

X_train_undersampling = X_train
y_train_undersampling = y_train
# print (y_train_undersampling)
print (X_train_undersampling.shape)

renn = RepeatedEditedNearestNeighbours (sampling_strategy='majority')
X_res, y_res = renn.fit_resample(X_train_undersampling, y_train_undersampling)
# print (y)
print (X_res.shape)

In [ ]:
dataset_res = []
for i in range(len(y_res)):
    dataset_res.append(np.append(X_res[i], y_res[i]))

dataset_res = np.asarray(dataset_res)

undersampled_df = pd.DataFrame(dataset_res, columns=columns)
undersampled_df

In [ ]:
undersampled_df['fraud'].value_counts()

In [ ]:
y_train.value_counts()

### Hybrid Approach

In [ ]:
from imblearn.combine import SMOTEENN

X_train_combine = X_train
y_train_combine = y_train

print (X_train_combine.shape)

sme = SMOTEENN(sampling_strategy='minority', ratio=1)
X_res, y_res = sme.fit_resample(X_train_combine, y_train_combine)

print (X_res.shape)

In [ ]:
dataset_res = []
for i in range(len(y_res)):
    dataset_res.append(np.append(X_res[i], y_res[i]))

dataset_res = np.asarray(dataset_res)

combined_df = pd.DataFrame(dataset_res, columns=columns)
combined_df

In [ ]:
combined_df['fraud'].value_counts()

## Model and Evaluation

In [ ]:
# from sklearn.model_selection import StratifiedKFold
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.metrics import accuracy_score

# X = oversampled_df.drop(columns=['fraud']).to_numpy()
# y = oversampled_df['fraud']

# model = MultinomialNB(alpha=1e-10)

# kf = StratifiedKFold(n_splits=10)
# for train_index, test_index in kf.split(X, y):
#     X_train_model, X_validation_model = X[train_index], X[test_index]
#     y_train_model, y_validation_model = y[train_index], y[test_index]
#     model.fit(X_train_model, y_train_model)
#     predict = model.predict(X_validation_model)
#     print ("Accuracy\t", accuracy_score(y_validation_model, predict))

In [ ]:
def show_history(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

In [ ]:
from sklearn.model_selection import StratifiedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

X = combined_df.drop(columns=['fraud']).to_numpy()
y = combined_df['fraud']

model = Sequential()
model.add(Dense(50, input_dim=16, kernel_initializer='uniform', activation='tanh'))
model.add(Dense(10, kernel_initializer='uniform', activation='relu'))
model.add(Dense(1, kernel_initializer='uniform', activation='relu'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'mae'])

kf = StratifiedKFold(n_splits=10)
for train_index, test_index in kf.split(X, y):
    X_train_model, X_validation_model = X[train_index], X[test_index]
    y_train_model, y_validation_model = y[train_index], y[test_index]
    history = model.fit(X_train_model, y_train_model, epochs=200, batch_size=10,  verbose=1)
    predict = model.predict_classes(X_validation_model)
    print ("Accuracy\t", accuracy_score(y_validation_model, predict))
    print(classification_report(y_validation_model, predict, target_names=['0', '1']))
    show_history(history)

In [ ]:
# from sklearn.metrics import classification_report

# predict_test = model.predict_classes(X_test)
# print ("Accuracy\t", accuracy_score(y_test, predict_test))
# print(classification_report(y_test, predict_test, target_names=['0', '1']))